<a href="https://colab.research.google.com/github/CaretJuice/gtm-sdr-generator/blob/main/Caret_Juice_Template_Auto_Populate_(All_Tags)_SDR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To use this, you will need to make a copy of this template so that you can edit where needed.

This is meant to run with a copy of the {{client name}} SDR template in Google Docs. You will need to create a sheet from that template, paste the URL of that sheet in the Variables section below and then select Run All in the Runtime dropdown in the primary navigation.

You will then need to manually upload an export of your GTM container in Upload File section.

Finally, you will need to give permissions for this notebook to make changes to Google Sheets in the Authenticate section.

Once that is done, this notebook will populate the Tags, Triggers, and Variables sections of the SDR template with no further action on your part.

This script completely replaces the text on those tabs.

GTM notes get copied to the Description columns. 

**Variables**

You will need to set these variables for this iPython notebook to work.

In [ ]:
spreadsheet = ''  # using the connect by URL method so put the full URL of the sheet that you are using for the SDR

# Configure which sheet tabs to place the data; these defaults match the SDR template at time of writing
tag_worksheet = 'Tags'
trigger_worksheet = 'Triggers'
variable_worksheet = 'Variables'

# Number of header rows
header_rows = 5



**Upload File**

Export the GTM Container under Admin > Export Container and upload the JSON File.

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving GTM-DGXW_v278.json to GTM-DGXW_v278.json


**Authenticate**

This will open a new window where you need to log in to Google Accounts, give this Colab permission to access Drive, copy the key that gets generated after giving permission, and finally paste that key back here and complete the authentication.

*2022-04-19: Updated authentication method*

In [ ]:
# authenticate
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

**Utilities** 

Utility functions that we are going to re-use later.

In [ ]:
%%capture
!pip install -U 'fsspec[s3]'

import json
import pandas as pd
from datetime import datetime
import re

uploaded_json = list(uploaded.values())[0]
container = json.loads(uploaded_json.decode('utf-8'))

json_tags = container['containerVersion']['tag']
json_triggers = container['containerVersion']['trigger']
json_folders = container['containerVersion']['folder']
json_variables = container['containerVersion']['variable']

def trigger_lookup(trigger_id_list):
  triggers = ''
  for trigger in json_triggers:
    for id in trigger_id_list:
      if id == trigger['triggerId']:
        triggers = triggers + trigger['name'] + ';'
  return triggers[:-1] #remove the last semi-colon

def folder_lookup(folder_id):
  name = 'not found'
  for folder in json_folders:
    if folder['folderId'] == folder_id:
      name = folder['name']
  return name

def list_tags_matching_trigger(trigger_type, trigger_id):
  tags = ''
  for tag in json_tags:
    if trigger_type in tag:
      if trigger_id in tag[trigger_type]: 
        tags = tags + tag['name'] + '; '
  return tags[:-2]


def timestamp_from_fingerprint(fingerprint):
  ts = datetime.fromtimestamp(int(fingerprint[:-3] ))
  return ts

def is_ga4_trigger( trigger_id ):
  is_ga4 = False
  for tag in json_tags:
    if (tag['type'] == 'gaawe' and trigger_id in tag['firingTriggerId']):
      is_ga4 = True 
  return is_ga4

def is_ga4_var( variable_name ):
  is_ga4 = False
  pattern = "{{" + variable_name + "}}"
  for tag in json_tags:
    if (tag['type'] == 'gaawe' and re.search(pattern, json.dumps(tag))):
      is_ga4 = True 
  return is_ga4

# do not use
def container_lookup(container_id):
  json_containers =  container['containerVersion']
  for container in json_containers:
    if container['containerId'] ==  container_id:
      return container['name']
    else:
      return container['containerId']

**Parse the Tags**

Drill down into the tag section of the JSON export. For each tag, find the values that correspond to the various columns in the spreadsheet.

Save the tag data to a dataframe and list the first five entries.

In [ ]:
tags = []

for tag in json_tags:
  row = {}
  row['ID'] = str(tag['tagId'])
  row['Name'] = str(tag['name'])
  row['Event Name'] = ''
  if 'parameter' in tag:
    for parameter in tag['parameter']:
      if (parameter['key'] == 'eventName' and 'value' in parameter):
        row['Event Name'] = parameter['value']
  row['Type'] =  str(tag['type'])
  if 'firingTriggerId' in tag:
    row['Firing Triggers'] = trigger_lookup(tag['firingTriggerId'])
  else: 
    row['Firing Triggers'] = 'none'
  if 'blockingTriggerId' in tag:
    row['Blocking Triggers'] =  trigger_lookup(tag['blockingTriggerId'])
  else: 
    row['Blocking Triggers'] = 'none'
  if 'parentFolderId' in tag:
    row['Parent Folder'] = folder_lookup(tag['parentFolderId'])
  else: 
    row['Parent Folder'] = 'none'
  row['Last Edit'] = timestamp_from_fingerprint(tag['fingerprint'])
  if 'paused' in tag:
    row['Paused'] = 'true'
  else: 
    row['Paused'] = 'false'
  row['Container'] = tag['containerId']
  row['Action'] = 'Review' # resetting the Action column because tags unlikely to match
  if 'notes' in tag:
    row['Description'] = tag['notes']
  else:
    row['Description'] = ""

  tags.append(row)

tag_data = pd.DataFrame(tags)
tag_data.head()

NameError: ignored

**Save to Google Sheets**

The will save as a GSheet in your Google Drive with the title of "GTM Tags for GTM-CONTAINERID." It will take a few minutes for the data to fully populate in the sheet so be patient.

In [ ]:
from gspread_dataframe import set_with_dataframe

ss = gc.open_by_url(spreadsheet)
ws = ss.worksheet(tag_worksheet)

#delete existing data
ws.clear()
set_with_dataframe(ws, tag_data, row=header_rows )
#restore the header text
ws.update('B3', 'Tags')

NameError: ignored

**Parse the Triggers**

Since the JSON export maps tags to triggers but not the reverse, we loop through all of the triggers and then for each trigger loop through all of the tags to get the tag names.

In [ ]:
triggers = []

for trigger in json_triggers:
  row = {}
  row['ID'] = str(trigger['triggerId'])
  row['Name'] = str(trigger['name'])
  row['Type'] =  str(trigger['type'])
  if 'filter' in trigger:
    row['Filter'] = str(trigger['filter'])
  else:
    row['Filter'] = 'none'
  if 'parentFolderId' in trigger:
    row['Parent Folder'] = folder_lookup(trigger['parentFolderId'])
  else: 
    row['Parent Folder'] = 'none'
  row['Firing Tags'] = list_tags_matching_trigger('firingTriggerId',trigger['triggerId']  );
  row['Blocking Tags'] = list_tags_matching_trigger('blockingTriggerId',trigger['triggerId']  );
  row['Last Edit'] = timestamp_from_fingerprint(trigger['fingerprint'])
  row['Action'] = 'Review' # resetting the Action column because tags unlikely to match
  row['Container'] = trigger['containerId']
  if 'notes' in trigger:
    row['Description'] = trigger['notes']
  else:
    row['Description'] = ''

  triggers.append(row)

trigger_data = pd.DataFrame(triggers)
trigger_data.head()

NameError: ignored

Save to Google Sheets

In [ ]:
ws = ss.worksheet(trigger_worksheet)

#delete existing data
ws.clear()
set_with_dataframe(ws, trigger_data, row=header_rows )
#restore the header text
ws.update('B3', 'Triggers')

NameError: ignored

Parse the Variables

In [ ]:
variables = []


for var in json_variables:
  row = {}
  row['ID'] = str(var['variableId'])
  row['Name'] = str(var['name'])
  row['Type'] =  str(var['type'])
  if 'parentFolderId' in var:
    row['Parent Folder'] = folder_lookup(var['parentFolderId'])
  else: 
    row['Parent Folder'] = 'none'
  row['Last Edit'] = timestamp_from_fingerprint(var['fingerprint'])
  row['Action'] = 'Review' # resetting the Action column because tags unlikely to match
  if 'notes' in var:
    row['Description'] = var['notes']
  else:
    row['Description'] = ""

  variables.append(row)

variable_data = pd.DataFrame(variables)
variable_data.head()

NameError: ignored

Save to Google Sheets

In [ ]:
ws = ss.worksheet(variable_worksheet)

#delete existing data
ws.clear()
set_with_dataframe(ws, variable_data, row=header_rows )
#restore the header text
ws.update('B3', 'Variables')

NameError: ignored